# Using Azure Monitor Logs extension in Azure Data Studio Insiders

Once Azure Monitor Logs (Preview) extension installed in Azure Data Studio Insiders build, you can connect to your Azure Monitor Log workspace(s), browse the tables, write/execute KQL queries against workspaces and write/execute Notebooks connected to the Azure Monitor Log kernel.

There are two main tables in Azure Log Analytics (Azure Monitor Logs) workspace that capture Azure SQL events:
1. AzureDiagnostics
2. AzureMetric


## 1\. Connect to Azure Monitor Logs (Log Analytics) workspace

Workspace is similar to what a database is to SQL. You connect to Log Analytics workspace to start querying data.

### 1.1 Install Azure Monitor Logs extension first

Go to the Extension viewlet and type in Azure Monitor Logs. Install it and restart ADS.

### 1.2 Connect to the desired Azure Monitor Logs workspace

Change the Kernel to "Log Analytics". Set Attach to to a new or existing connection to the workspace. Note: you will need a workspace Id that you can obtain from Azure portal.

> **Note**: the name of the kernel is subject to change.

## 2. Analyze events by Diagnostic Settings

Let's do a simple query first to analyze the number of events by Operation Name. 

> **Note**: Each row in AzureDiagnostic represents an event for specific Operation or category. Some SQL actions may result in generating multiple events of different types.



In [33]:
AzureDiagnostics
| summarize count() by OperationName

Commands completed successfully.

Total execution time: 00:00:00.365

OperationName,count_
AuditEvent,4249
DatabaseWaitStatistcsEvent,64
QueryStoreRuntimeStatisticsEvent,119
DeadlockEvent,3
ErrorEvent,108
TimeoutEvent,4
AutomaticTuningSettingsSnapshotEvent,4


The above query's equivalent in SQL is:

```
SELECT OperationName, COUNT(*) AS [count_]
FROM AzureDiagnostics
GROUP BY OperationName
```

Count my Azure SQL DB events by category / diagnostic settings.

In [34]:
AzureDiagnostics
| where LogicalServerName_s == "jukoesmasqldb"
// | where TimeGenerated >= ago(5d)
| summarize count() by Category


Commands completed successfully.

Total execution time: 00:00:00.231

Category,count_
DatabaseWaitStatistics,64
SQLSecurityAuditEvents,4248
QueryStoreRuntimeStatistics,119
Deadlocks,3
Errors,108
Timeouts,4
AutomaticTuning,4
DevOpsOperationsAudit,1


## 3. Performance troubleshooting Query (from Azure Portal)

Potentially a query or deadlock on the system that could lead to poor performance. The following is a query suggested by Azure Portal.

In [35]:
AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(12d)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName

Commands completed successfully.

Total execution time: 00:00:00.049

Resource,MetricName,Deadlock_max_60Mins
jukoesmasqldb/databases/adventureworks,deadlock,1
jukoesmasqldb/databases/wideworldimporters,deadlock,1


# AzureMetrics

This is a sample query to dig into AzureMetrics

In [36]:
AzureMetrics
| project-away TenantId, ResourceId, SubscriptionId, _ResourceId, ResourceGroup // hide sensitive info
| project TimeGenerated, MetricName, Total, Count, UnitName
| take 10

Commands completed successfully.

Total execution time: 00:00:00.562

TimeGenerated,MetricName,Total,Count,UnitName
2021-07-28 07:42:00,sessions_percent,0,4,Percent
2021-07-28 07:42:00,allocated_data_storage,1442840576,2,Bytes
2021-07-28 07:42:00,log_write_percent,0,4,Percent
2021-07-28 07:42:00,workers_percent,0,4,Percent
2021-07-28 07:42:00,allocated_data_storage,33554432,1,Bytes
2021-07-28 07:42:00,workers_percent,0,4,Percent
2021-07-28 07:42:00,storage_percent,0,1,Percent
2021-07-28 07:42:00,cpu_percent,0,4,Percent
2021-07-28 07:42:00,cpu_limit,10,5,Count
2021-07-28 07:42:00,cpu_percent,0,4,Percent


# AzureDiagnostics

This is a sample query to dig into AzureDiagnostics. This table tends to have more details than AzureMetrics.

In [37]:
AzureDiagnostics
| project-away TenantId, ResourceId, SubscriptionId, ResourceGroup, _ResourceId // Hide sensitive columns :) 
| project TimeGenerated, Category, OperationName
| take 10

Commands completed successfully.

Total execution time: 00:00:00.304

TimeGenerated,Category,OperationName
2021-07-08 20:22:25,SQLSecurityAuditEvents,AuditEvent
2021-07-08 20:55:53,QueryStoreRuntimeStatistics,QueryStoreRuntimeStatisticsEvent
2021-07-08 22:04:55,SQLSecurityAuditEvents,AuditEvent
2021-07-08 22:04:55,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:10:49,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:10:13,QueryStoreRuntimeStatistics,QueryStoreRuntimeStatisticsEvent
2021-07-08 21:02:12,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:25:06,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:25:06,SQLSecurityAuditEvents,AuditEvent
2021-07-08 22:15:35,SQLSecurityAuditEvents,AuditEvent


## Analyze (non-audit) Events

In [38]:
AzureDiagnostics
| summarize event_count = count() by bin(TimeGenerated, 2d), OperationName
| where OperationName <> "AuditEvent"
| evaluate pivot(OperationName, sum(event_count))
| sort by TimeGenerated asc

Commands completed successfully.

Total execution time: 00:00:00.395

TimeGenerated,AutomaticTuningSettingsSnapshotEvent,DatabaseWaitStatistcsEvent,DeadlockEvent,ErrorEvent,QueryStoreRuntimeStatisticsEvent,TimeoutEvent
2021-06-29 17:00:00,0,1,0,2,2,0
2021-07-01 17:00:00,0,0,0,0,2,0
2021-07-05 17:00:00,0,0,0,0,1,0
2021-07-07 17:00:00,0,0,0,0,2,0
2021-07-13 17:00:00,0,4,0,0,9,0
2021-07-15 17:00:00,0,0,1,28,4,0
2021-07-17 17:00:00,0,4,1,21,11,0
2021-07-19 17:00:00,4,32,0,57,38,4
2021-07-21 17:00:00,0,0,0,0,39,0
2021-07-25 17:00:00,0,0,0,0,7,0


In [39]:
AzureDiagnostics
| summarize event_count=count() by bin(TimeGenerated, 2d), OperationName
// | where OperationName <> "AuditEvent"

Commands completed successfully.

Total execution time: 00:00:00.254

TimeGenerated,OperationName,event_count
2021-07-27 17:00:00,DatabaseWaitStatistcsEvent,23
2021-07-27 17:00:00,AuditEvent,546
2021-07-27 17:00:00,QueryStoreRuntimeStatisticsEvent,4
2021-07-27 17:00:00,DeadlockEvent,1
2021-07-01 17:00:00,AuditEvent,9
2021-07-01 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-07-05 17:00:00,QueryStoreRuntimeStatisticsEvent,1
2021-07-05 17:00:00,AuditEvent,86
2021-07-19 17:00:00,AuditEvent,1747
2021-07-19 17:00:00,DatabaseWaitStatistcsEvent,32


In [40]:
AzureDiagnostics
| make-series event_count = count() on TimeGenerated from datetime(2021-07-20 22:00:00) to now() step 1m   
| render timechart


Commands completed successfully.

Total execution time: 00:00:00.750

event_count,TimeGenerated


## Deadlock Analysis

In [41]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| project TimeGenerated, Category, Resource, OperationName, Type, deadlock_xml_s
| sort by TimeGenerated desc
| take 50

Commands completed successfully.

Total execution time: 00:00:00.397

TimeGenerated,Category,Resource,OperationName,Type,deadlock_xml_s
2021-07-28 08:27:27,Deadlocks,WIDEWORLDIMPORTERS,DeadlockEvent,AzureDiagnostics,<deadlock> <victim-list> <victimProcess id='process1be4418eca8'/> </victim-list> <process-list> <process id='process1be4418eca8' taskpriority='0' logused='384' waitresource='KEY: 5:72057594063290368 (5495ec521809)' waittime='15671' ownerId='5068976' transactionname='user_transaction' lasttranstarted='2021-07-28T15:27:12.197' XDES='0x1be55b24428' lockMode='U' schedulerid='2' kpid='182112' status='suspended' spid='80' sbid='0' ecid='0' priority='0' trancount='2' lastbatchstarted='2021-07-28T15:27:12.193' lastbatchcompleted='2021-07-28T15:27:12.147' lastattention='1900-01-01T00:00:00.147' clientapp='azdata-Query' hostname='filtered' hostpid='4660' loginname='filtered' isolationlevel='read committed (2)' xactid='5068976' currentdb='5' currentdbname='WideWorldImporters' lockTimeout='4294967295' clientoption1='671156320' clientoption2='128056'> <executionStack> <frame procname='unknown' queryhash='0xfb56c41f149be1bf' queryplanhash='0x61108d8e53ae0281' line='7' stmtstart='236' stmtend='398' sqlhandle='0x0200000076fe5d1c0c6a574e5832f3b8ff93e633e3a1d6ae0000000000000000000000000000000000000000'> unknown </frame> <frame procname='unknown' queryhash='0xfb56c41f149be1bf' queryplanhash='0x61108d8e53ae0281' line='7' stmtstart='236' stmtend='398' sqlhandle='0x0200000005a1aa314a4b357187cf390d3dc4890a0bd7a7c10000000000000000000000000000000000000000'> unknown </frame> </executionStack> <inputbuf> filtered </inputbuf> </process> <process id='process1be4779e108' taskpriority='0' logused='572' waitresource='KEY: 5:72057594063224832 (8194443284a0)' waittime='4814' ownerId='5068689' transactionname='user_transaction' lasttranstarted='2021-07-28T15:27:08.060' XDES='0x1be55b20428' lockMode='U' schedulerid='2' kpid='29248' status='suspended' spid='68' sbid='0' ecid='0' priority='0' trancount='2' lastbatchstarted='2021-07-28T15:27:08.037' lastbatchcompleted='2021-07-28T15:26:58.057' lastattention='1900-01-01T00:00:00.057' clientapp='azdata-Query' hostname='filtered' hostpid='4660' loginname='filtered' isolationlevel='read committed (2)' xactid='5068689' currentdb='5' currentdbname='WideWorldImporters' lockTimeout='4294967295' clientoption1='671156320' clientoption2='128056'> <executionStack> <frame procname='unknown' queryhash='0xa57159d12cb813fb' queryplanhash='0x10cbf3310dab2eb5' line='9' stmtstart='58' stmtend='186' sqlhandle='0x020000009897910214549261257b982dc5f67c7e4f562ee10000000000000000000000000000000000000000'> unknown </frame> <frame procname='unknown' queryhash='0xa57159d12cb813fb' queryplanhash='0x10cbf3310dab2eb5' line='9' stmtstart='382' stmtend='520' sqlhandle='0x020000003cbaf1075f6082bedf3e15172fe9ecb6c23b78160000000000000000000000000000000000000000'> unknown </frame> </executionStack> <inputbuf> filtered </inputbuf> </process> </process-list> <resource-list> <keylock hobtid='72057594063290368' dbid='5' objectname='filtered' indexname='filtered' id='lock1be5c46dc00' mode='X' associatedObjectId='72057594063290368'> <owner-list> <owner id='process1be4779e108' mode='X'/> </owner-list> <waiter-list> <waiter id='process1be4418eca8' mode='U' requestType='wait'/> </waiter-list> </keylock> <keylock hobtid='72057594063224832' dbid='5' objectname='filtered' indexname='filtered' id='lock1be5c403000' mode='X' associatedObjectId='72057594063224832'> <owner-list> <owner id='process1be4418eca8' mode='X'/> </owner-list> <waiter-list> <waiter id='process1be4779e108' mode='U' requestType='wait'/> </waiter-list> </keylock> </resource-list> </deadlock>
2021-07-19 06:55:22,Deadlocks,ADVENTUREWORKS,DeadlockEvent,AzureDiagnostics,<deadlock> <victim-list> <victimProcess id='process1e9a7b324e8'/> </victim-list> <process-list> <process id='process1e9a7b324e8' taskpriority='0' logused='388' waitresource='KEY: 5:72057594047037440 (5495ec521809)' waittime='15032' ownerId='28766101' transactionname='user_transaction' lasttr

Find the deadlock query plan

In [42]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| extend d = parse_xml(deadlock_xml_s)
| project TimeGenerated, QueryPlanHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryplanhash"], QueryHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryhash"]
| take 50

Commands completed successfully.

Total execution time: 00:00:01.051

TimeGenerated,QueryPlanHash,QueryHash
2021-07-19 06:55:22,0x61108d8e53ae0281,0xfb56c41f149be1bf
2021-07-17 16:13:34,0x61108d8e53ae0281,0xfb56c41f149be1bf
2021-07-28 08:27:27,0x61108d8e53ae0281,0xfb56c41f149be1bf


## Query Store Runtime Statistics Events

In [43]:
AzureDiagnostics
| where OperationName == "QueryStoreRuntimeStatisticsEvent"
| project TimeGenerated, query_hash_s, statement_sql_handle_s, query_plan_hash_s
| take 10

Commands completed successfully.

Total execution time: 00:00:00.477

TimeGenerated,query_hash_s,statement_sql_handle_s,query_plan_hash_s
2021-07-20 19:30:02,0x4E8C0B5AF78C4ED1,0x0900C6D4FC50EF5B4E298DBE4E3A7EA7911F0000000000000000000000000000000000000000000000000000,0xB44B4BF819B82F62
2021-07-20 19:30:02,0xE8E6F7DC2D2909FE,0x0900B90F6497028D7C84720EFD387ECE8C2F0000000000000000000000000000000000000000000000000000,0x037E2E7A34FBE1AD
2021-07-21 12:58:35,0xD47FA8AE1E407243,0x0900B10677BE63CAE65A693887D6502113BC0000000000000000000000000000000000000000000000000000,0xFA26A5F6A84C8679
2021-07-21 12:58:35,0xD0B172EC3AC90AB2,0x0900F0816A25DA2ADDAACB957F82B882EC160000000000000000000000000000000000000000000000000000,0x2B9A3A3B55B4B6CB
2021-07-21 13:48:36,0xE9A90E6DF072038B,0x09003A8994953FDD36568246F07679879BA60000000000000000000000000000000000000000000000000000,0x034C49FD4D7CD726
2021-07-21 13:48:36,0x1952171B581C894D,0x0900506DB841E46B2920275FF14C13E8BFC70000000000000000000000000000000000000000000000000000,0xB496791006554667
2021-07-21 13:48:36,0x1952171B581C894D,0x0900FA2BFA7C760AF0B2E9D5ED64A043458D0000000000000000000000000000000000000000000000000000,0xF3695F01D8FE0E91
2021-07-21 13:48:36,0x5D10B550067140CF,0x0900C2E90943F6588EBCEBF060DC800B68AB0000000000000000000000000000000000000000000000000000,0x754D0416154A71D1
2021-07-21 13:48:36,0x82C4887635CEB06D,0x09006EFFD14D9A2EA5112AD9C2FCDB049B200000000000000000000000000000000000000000000000000000,0xA0268D84A1CF355E
2021-07-21 13:48:36,0xADCF2ECC6A4FE6E4,0x090048201C1EC9E32104315E4CD4D45627F90000000000000000000000000000000000000000000000000000,0x54B4226B7D07F982


## Analyze Errors

In [44]:
AzureDiagnostics
| where OperationName == "ErrorEvent"
| extend ErrorNumber =  tostring(error_number_d) 
| summarize event_count=count() by EventTime = bin(TimeGenerated, 2d),  ErrorNumber
| evaluate pivot(ErrorNumber, sum(event_count))
| sort by EventTime asc


Commands completed successfully.

Total execution time: 00:00:00.344

EventTime,1222.0,208.0,2714.0,3902.0
2021-06-29 17:00:00,0,2,0,0
2021-07-15 17:00:00,0,27,0,1
2021-07-17 17:00:00,0,20,0,1
2021-07-19 17:00:00,8,47,1,1


## Find Deleted table

In [45]:
AzureDiagnostics
| where action_name_s in ('BATCH COMPLETED')
| project TimeGenerated, Category, action_name_s, statement_s
| where statement_s contains "DROP TABLE"
| sort by TimeGenerated desc 
| take 10



Commands completed successfully.

Total execution time: 00:00:00.451

TimeGenerated,Category,action_name_s,statement_s
2021-07-20 19:26:55,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:26:55,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:26:15,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:26:15,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:24:17,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:24:17,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:22:25,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-20 19:22:25,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-19 06:55:35,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
2021-07-19 06:55:35,SQLSecurityAuditEvents,BATCH COMPLETED,DROP TABLE [dbo].[Person] DROP TABLE [dbo].[PersonPhone]
